# Simulating a virtual Soft X-ray Tomography system

In [ ]:
# imports
from utils import *
from common import *
from scipy.stats import lognorm

In [2]:
# parameters
KEEP_ONLY_TO_KEEP = True # [bool] keep only the rays kept by Luca O. in the real dataset
# dataset parameters
# from common import SXRV_SIZE, SXRH_SIZE
# SXRV_SIZE = 21 # number of vertical rays
# SXRH_SIZE = 23 # number of horizontal rays
# CV, CH = (2.0, -0.7), (2.7, 0) # centers of the fans
# MAXVANGLE = π/5 # maximum vertical angle
# MAXHANGLE = π/5 # maximum horizontal angle
POLAR = False # use polar coordinates

# constants for random gaussians
MAX_MIX = 1 # max number of gaussians
KR = (0.1, 0.8) if POLAR else (0.0, 0.9)  # [m] max radius mean multiplier
KXY = (1/10, 1/4) # std deviation range multiplier
# KMIX = (0.2, 1.0)  # min and max constant for mixing gaussians # KMIX sobstituted by sampling from a lognormal distribution (see MAX_EMISS_LOGNORM_PARAMS)

# N = 500_000 # train
# N = 100_000 # train
N = 50_000 # train
# N = 10_000 # train

In [3]:
# helper functions
def wrap_angle(α): return np.arctan2(np.sin(α), np.cos(α))

def gaussian(v, μ=np.array([R0+L/2, Z0+L/2]), Σ=np.array([[L/4,0],[0,L/4]]), polar=False):
    rshape = v.shape[:-1] # save the original shape
    v = v.reshape(-1, 2) # flatten the input to 2D
    d = v-μ # difference vector
    if polar: d[:,1] = wrap_angle(d[:,1]) 
    g = np.exp(-0.5*np.sum(d @ inv(Σ) * d, axis=-1)) # gaussian formula
    r = g.reshape(rshape) # return the result in the original shape
    return r

# #fake gaussian
# def gaussian(v, μ=np.array([R0+L/2, Z0+L/2]), Σ=np.array([[L/4,0],[0,L/4]]), polar=False):
#     rshape = v.shape[:-1] # save the original shape
#     v = v.reshape(-1, 2) # flatten the input to 2D
#     d = v-μ # difference vector
#     if polar: d[:,1] = wrap_angle(d[:,1]) 
#     g = np.sqrt(0.5*np.sum(d @ inv(Σ) * d, axis=-1)) # fake gaussian formula
#     r = g.reshape(rshape) # return the result in the original shape
#     return r

def create_line(c, θ, n=10*RES): # create a line from a center and an absolute angle
    cθ, sθ = cos(θ), sin(θ)
    if np.abs(cθ) > np.abs(sθ): # less than 45 degrees
        x = np.linspace(R0-δ/2, R1+δ/2, n)
        y = (sθ/cθ)*x + (c[1] - (sθ/cθ)*c[0])
    else: # more than 45 degrees
        y = np.linspace(Z0-δ/2, Z1+δ/2, n)
        x = (cθ/sθ)*y + (c[0] - (cθ/sθ)*c[1])
    # keep only points inside the grid/first wall
    # idxs = (R0-δ/2 <= x) & (x <= R1+δ/2) & (Z0-δ/2 <= y) & (y <= Z1+δ/2) # inside grid
    idxs = (x-RM)**2 + (y-ZM)**2 <= (R_FW+δ/2)**2 # inside first wall #TODO: not exact Radius FW < L/2
    if sum(idxs) == 0: print(f'Warning: line outside: c={c}, θ={θ:.2f}')
    return np.stack((x[idxs], y[idxs]), axis=-1)

def line_mask(c, θn, θl, n=16*RES): # mask for a line, c=center, θn=normal angle, θl=angle of the line wrt the normal
    lin = create_line(c, θn+θl, n)
    mask = np.zeros((RES, RES)).reshape(-1)
    grid = RZ.copy().reshape(-1, 2)
    # for l in lin: mask[np.argmin(norm(grid-l, axis=-1))] += RES/n # easy way
    for l in lin: # more accurate way (still extremely inefficient)
        d = norm(grid-l, axis=-1) # get the distances
        idxs = np.argsort(d)[:4] #get the n closest points
        d = d[idxs] #get the  distances
        w = 1/d #get the weights
        w /= w.sum() #normalize the weights
        mask[idxs] += w*RES/n #add the weights to the mask
    mask_idxs = np.where(mask > 0)
    mask = mask[mask_idxs]
    mask = mask*cos(θl)#**2 # the inclination of the line reduces the mask by cos(θl) #TODO: check if this is correct
    return mask, mask_idxs

In [4]:
# convert from polar to cartesian coordinates and vice versa
def polar2cart(v): 
    vshape = v.shape
    v = v.reshape(-1, 2)
    r, θ = v[:,0], v[:,1]
    x, y = RM+r*cos(θ), ZM+r*sin(θ)
    xy = np.stack((x, y), axis=-1)
    return xy.reshape(vshape)

def cart2polar(v): 
    vshape = v.shape
    v = v.reshape(-1, 2)
    x, y = v[:,0], v[:,1]
    r, θ = hypot(x-RM, y-ZM), arctan2(y-ZM, x-RM)
    rθ = np.stack((r, θ), axis=-1)
    return rθ.reshape(vshape)

In [ ]:
# test polar2cart and cart2polar
RZ_POL = cart2polar(RZ)
assert RZ.shape == RZ_POL.shape, f'Expected {RZ.shape} but got {RZ_POL.shape}'
assert np.allclose(RZ_POL, cart2polar(polar2cart(RZ_POL))), 'cart2polar(polar2cart(x)) != x'
assert np.allclose(RZ, polar2cart(cart2polar(RZ))), 'polar2cart(cart2polar(x)) != x'
# plot RZ_POL
plt.figure(figsize=(6,6))
plt.scatter(*RZ_POL.T, s=5, c='r')
plt.title('RZ_POL')
plt.xlabel('R')
plt.ylabel('Z')
plt.show()

In [ ]:
# test line
c, θ = (2, 0), π/2
c, θ = (2, 0), uniform(0, π)
# c, θ = (2, 0), π/6

l1 = create_line(c, θ)
print(l1.shape)

mask, midxs = line_mask(c, 0, θ)
print(np.sum(mask))

square_mask = np.zeros((RES*RES))
square_mask[midxs] = mask

# plot
plt.figure()
# plt.scatter(l1[:,0], l1[:,1], c='r', s=1)
plt.scatter(RRH, ZZH, c=square_mask.reshape((RES,RES)), s=10, cmap='gray')
plt.plot(l1[:,0], l1[:,1], ':y')
plt.plot(FW[:,0], FW[:,1], 'w')
plt.xlim(R0-δ/2, R1+δ/2)
plt.ylim(Z0-δ/2, Z1+δ/2)
plt.gca().set_aspect('equal', adjustable='box')
plt.grid(False)
plt.colorbar()
plt.show()

In [ ]:
# test polar gaussian and rays
# #standard gaussian
# μ = np.array([R0+2*L/3, Z0+3*L/5])
# Σ = np.array([[L/40, 0], [0, L/16]])
# gauss = gaussian(RZ, μ, Σ) 

# polar gaussian
# μ, Σ = np.array([0.25, π/3]), np.array([[1/100, 0.0], [0.0, π/3]])
# μ, Σ = np.array([0.0, π/3]), np.array([[1/500, 0.0], [0.0, 100*π]])
μ, Σ = np.array([0.2, π/3]), np.array([[1/500, 0.0], [0.0, π/5]])
gauss_pol = gaussian(RZ_POL, μ, Σ, polar=True)
gauss = gauss_pol
# c, θ = (2, 0), π/6
# c, θ = (2, 0), uniform(0, π)
c, θ = (uniform(1.5,2.5), uniform(-.5, .5)), uniform(0, π)
mask1, midxs1 = line_mask(c, 0, θ)
square_mask1 = np.zeros((RES*RES))
square_mask1[midxs1] = mask1

combined = gauss.reshape(-1)[midxs1] * mask1 * δ

combined_square = np.zeros((RES*RES))
combined_square[midxs1] = combined

# sxr integration values
sxr = np.sum(combined)
print(f'SXR: {sxr:.4f}')

# plot the 4 maps
#plot gasussian polar
plt.figure(figsize=(20,5))
plt.subplot(141)
plt.scatter(*RZ_POL.T, s=5, c=gauss_pol)
plt.title('Gaussian Polar')
plt.xlabel('r'), plt.ylabel('θ')
plt.colorbar()
plt.subplot(142)
plt.scatter(RRH, ZZH, c=gauss, s=20)
plt.clim(0, 1)
plt.title('Gaussian')
plt.axis('equal')
plt.colorbar()
plt.grid(False)
plt.xlim(R0, R1), plt.ylim(Z0, Z1)
plt.subplot(143)
plt.scatter(RRH, ZZH, c=square_mask1, s=10)
plt.clim(0, 1)
plt.title('Line')
plt.axis('equal')
plt.colorbar()
plt.grid(False)
plt.xlim(R0, R1), plt.ylim(Z0, Z1)
plt.subplot(144)
plt.scatter(RRH, ZZH, c=combined_square/δ, s=10)
plt.clim(0, 1)
plt.title('Combined')
plt.axis('equal')
plt.colorbar()
plt.grid(False)
plt.xlim(R0, R1), plt.ylim(Z0, Z1)
plt.show()

In [ ]:
# create RFX fans of rays (takes a while)
μ = np.array([R0+2*L/3, Z0+3*L/5])
Σ = np.array([[L/16, 0], [0, L/40]])
gauss = gaussian(RZ, μ, Σ) 
names = ['VDI', 'VDC', 'VDE', 'HOR']
nrays = [VDI_NRAYS, VDC_NRAYS, VDE_NRAYS, HOR_NRAYS]
αs_start = [VDI_START_ANGLE, VDC_START_ANGLE, VDE_START_ANGLE, HOR_START_ANGLE]
αs_span = [VDI_SPAN_ANGLE, VDC_SPAN_ANGLE, VDE_SPAN_ANGLE, HOR_SPAN_ANGLE]
pinholes = [VDI_PINHOLE_POS, VDC_PINHOLE_POS, VDE_PINHOLE_POS, HOR_PINHOLE_POS]
colors = ['r', 'g', 'b', 'y']
sxrs, rayss, fans, inc_αs = [], [], [], [] # sxr, rays, fans, incidence angles
for name, n, α_start, α_span, p, c in zip(names, nrays, αs_start, αs_span, pinholes, colors):
    α_normal = α_start+α_span/2 # normal incidence angle of the fan
    αs_rays = np.linspace(α_start, α_start+α_span, n) # absolutre angles of the rays
    αs_incidence = αs_rays - α_normal # angles of incidence wrt the normal
    inc_αs.append(αs_incidence)
    rays = [create_line(p, αr) for αr in αs_rays]
    fan = [line_mask(p, α_normal, αi) for αi in αs_incidence]
    combined = np.zeros((n, RES*RES))
    for i, (mask, midxs) in enumerate(fan):
        combined[i,midxs] = gauss.reshape(-1)[midxs] * mask * δ
    combined_square = np.zeros((n, RES*RES))
    for i, (mask, midxs) in enumerate(fan):
        combined_square[i,midxs] = combined[i,midxs]
    sxr = np.sum(combined, axis=-1)
    rayss.append(rays)
    fans.append(fan)
    sxrs.append(sxr)
    plt.figure(figsize=(15,3))
    plt.subplot(131)
    plt.scatter(RRH, ZZH, c=gauss, s=20)
    plt.clim(0, 1)
    for r in rays: plt.plot(r[:,0], r[:,1], f'{c}')
    plt.axis('equal')
    plt.xlim(R0, R1), plt.ylim(Z0, Z1)
    plt.colorbar()
    plt.title(f'{name} Rays')
    plt.grid(False)
    plt.subplot(132)
    plt.scatter(RRH, ZZH, c=combined_square.sum(axis=0)/δ, s=5)
    plt.axis('equal')
    plt.clim(0, 1)
    plt.xlim(R0, R1), plt.ylim(Z0, Z1)
    plt.grid(False)
    plt.colorbar()
    plt.title(f'{name} Rays integration')
    plt.subplot(133)
    plt.plot(αs_incidence, sxr, f'{c}s-')
    plt.xticks([-π/4, -π/8, 0, π/8, π/4], ['-π/4', '-π/8', '0', 'π/8', 'π/4'])
    plt.grid(True)
    plt.ylim(0, 1)
    plt.title(f'{name} SXR')
    plt.show()

if KEEP_ONLY_TO_KEEP:
    # KEEP ONLY THE IDXS TO KEEP (SEE common.py)
    to_keeps = [VDI_TO_KEEP, VDC_TO_KEEP, VDE_TO_KEEP, HOR1_TO_KEEP] # NOTE: ignore HOR2_TO_KEEP for now, handle it later in the dataset
    for i in range(4):
        rayss[i] = [rayss[i][j] for j in to_keeps[i]]
        fans[i] = [fans[i][j] for j in to_keeps[i]]
        sxrs[i] = sxrs[i][to_keeps[i]]
        inc_αs[i] = inc_αs[i][to_keeps[i]]

# plot everything together
plt.figure(figsize=(18,8))
plt.subplot(121)
plt.scatter(RRH, ZZH, c=gauss, s=20)
plt.clim(0, 1)
for rs, c in zip(rayss, colors):
    for r in rs: plt.plot(r[:,0], r[:,1], f'{c}')
plt.axis('equal')
plt.xlim(R0, R1), plt.ylim(Z0, Z1)
plt.colorbar()
plt.title('Rays')
plt.grid(False)
plt.subplot(122)
for θs, c, sxr, s, α, in zip(inc_αs, colors, sxrs, αs_start, αs_span):
    plt.plot(θs, sxr, f'{c}s-')
plt.xticks([-π/4, -π/8, 0, π/8, π/4], ['-π/4', '-π/8', '0', 'π/8', 'π/4'])
plt.grid(True)
plt.ylim(0, 1)
plt.title('SXR')
plt.show()

# Create a Dataset

In [9]:
# helper functions
def create_random_means_stds(max_mix=MAX_MIX, kr=KR, kxy=KXY, polar=POLAR):
    n = np.random.randint(1, max_mix+1) if max_mix > 1 else 1 # number of gaussians
    # mix = uniform(kmix[0], kmix[1], n) # mixing coefficients
    mix = lognorm.rvs(*MAX_EMISS_LOGNORM_PARAMS, size=n) # mixing coefficients
    μr = uniform(L*kr[0]/2, L*kr[1]/2, n) # [m] random radius
    μθ = uniform(0, 2*π, n) # [rad] random angle
    μ = np.stack((μr, μθ), axis=-1) # [m, rad] random mean
    Σ = np.zeros((n, 2, 2)) # covariance matrix
    if polar:
        # kθ = (μr*kxy[0], μr*kxy[1]) # [rad] random angle std deviation
        # Σ[:,1,1] = uniform(kθ[0], kθ[1], n)**2
        Σ[:,0,0] = uniform(L*kxy[0], L*kxy[1], n)**2
        Σ[:,1,1] = uniform(L*2*π*kxy[0], L*2*π*kxy[1], n)**2
    else:
        μ = polar2cart(μ) #convert to cartesian 
        Σ[:,0,0] = uniform(L*kxy[0], L*kxy[1], n)**2
        Σ[:,1,1] = uniform(L*kxy[0], L*kxy[1], n)**2
    return mix, μ, Σ

def eval_gaussians(ps, mix, μ, Σ, polar=False):
    assert len(mix) == len(μ) == len(Σ), 'mix, μ, Σ must be lists of the same length'
    d = np.zeros(ps.shape[:-1]) # initialize the distribution
    if polar: ps = cart2polar(ps) # convert to polar
    for i in range(len(μ)): d += mix[i]*gaussian(ps, μ[i], Σ[i], polar) # add the gaussians
    return d

def create_random_gaussian_mix():
    mix, μ, Σ = create_random_means_stds()
    g = eval_gaussians(RZ, mix, μ, Σ, POLAR)
    return g

def eval_on_fan(d, f): # d distribution, f fan: [(mask, mask indexes), ...]
    sxr = np.zeros(len(f)) # sxr integration values
    assert d.shape[0] == d.shape[1], 'distribution must be square'    
    for i, (m, mi) in enumerate(f): # for each ray, m: mask, mi: mask indexes
        sxr[i] = np.sum(d.reshape(-1)[mi]*m*δ) # integrate the distribution along the ray
    return sxr

In [ ]:
# test
mix, μ, Σ = create_random_means_stds(MAX_MIX, KR, KXY, POLAR) # create random means and stds
g = eval_gaussians(RZ, mix, μ, Σ, POLAR) # evaluate the gaussians
# sxrh = eval_on_fan(g, HFAN)
# sxrv = eval_on_fan(g, VFAN)
sxrs = [eval_on_fan(g, f) for f in fans] # evaluate the SXR on the fans
g_lr = g[::KHRES, ::KHRES] # low resolution distribution

# plot g and sxrh
plt.figure(figsize=(15,5))
plt.subplot(131)
plt.scatter(RRH, ZZH, c=g, s=20)
# plot rays
for rs, c in zip(rayss, colors):
    for r in rs: plt.plot(r[:,0], r[:,1], f'{c}')
plt.axis('equal')
plt.xlim(R0, R1), plt.ylim(Z0, Z1)
plt.colorbar()
plt.title('Distribution HR')
plt.grid(False)
plt.subplot(132)
plt.scatter(RRL, ZZL, c=g_lr, s=20)
plt.axis('equal')
plt.xlim(R0, R1), plt.ylim(Z0, Z1)
plt.colorbar()
plt.title('Distribution LR')
plt.grid(False)
plt.subplot(133)
for c, sxr, a in zip(colors, sxrs, inc_αs):
    plt.plot(a, sxr, f'{c}s-')
plt.grid(True)
plt.legend()
plt.title('SXR')
plt.show()

In [ ]:
# create dataset (N train samples, N//10 test/validation samples)
def create_dataset(n):
    emiss_hr = np.zeros((n, RES, RES), dtype=np.float32) # emissivity high resolution
    emiss_lr = np.zeros((n, RES//KHRES, RES//KHRES), dtype=np.float32) # emissivity low resolution
    vdi = np.zeros((n, len(fans[0]),), dtype=np.float32) 
    vdc = np.zeros((n, len(fans[1]),), dtype=np.float32) 
    vde = np.zeros((n, len(fans[2]),), dtype=np.float32)
    hor = np.zeros((n, len(fans[3]),), dtype=np.float32)
    for i in tqdm(range(n), desc=f'Creating {n} samples'):
        mix, μ, Σ = create_random_means_stds(MAX_MIX, KR, KXY, POLAR)
        emiss_hr[i] = eval_gaussians(RZ, mix, μ, Σ, POLAR)
        vdi[i] = eval_on_fan(emiss_hr[i], fans[0])
        vdc[i] = eval_on_fan(emiss_hr[i], fans[1])
        vde[i] = eval_on_fan(emiss_hr[i], fans[2])
        hor[i] = eval_on_fan(emiss_hr[i], fans[3])
        emiss_lr[i] = resize2d(emiss_hr[i], (RES//KHRES, RES//KHRES))
    # save the dataset
    np.savez(f'data/sxr_sim_ds_{n}.npz', emiss_hr=emiss_hr, emiss_lr=emiss_lr, vdi=vdi, vdc=vdc, vde=vde, hor=hor, RRH=RRH, ZZH=ZZH, RRL=RRL, ZZL=ZZL)

create_dataset(N)
create_dataset(N//10)
# create_dataset(N//100)

In [ ]:
# load the dataset
data = np.load(f'data/sxr_sim_ds_{N//10}.npz')
print(data.files)
# extract the data
emiss_hr, emiss_lr, vdi, vdc, vde, hor = data['emiss_hr'], data['emiss_lr'], data['vdi'], data['vdc'], data['vde'], data['hor']
print(emiss_hr.shape, emiss_lr.shape, vdi.shape, vdc.shape, vde.shape, hor.shape)

In [ ]:
# plot the dataset
N_PLOTS = 10
idxs = np.random.randint(0, N//10, N_PLOTS)
for i in idxs:
    plt.figure(figsize=(15,3))
    mind, maxd = np.min(emiss_hr[i]), np.max(emiss_hr[i])
    # distribution
    plt.subplot(131)
    plt.scatter(RRH, ZZH, c=emiss_hr[i], s=6)
    plt.axis('equal')
    plt.xlim(R0, R1), plt.ylim(Z0, Z1)
    plt.colorbar()
    plt.clim(mind, maxd)
    plt.title(f'Distribution {i}')
    plt.grid(False)
    # subgrid
    plt.subplot(132)
    plt.scatter(RRL, ZZL, c=emiss_lr[i], s=3)
    plt.axis('equal')
    plt.xlim(R0, R1), plt.ylim(Z0, Z1)
    plt.colorbar()
    plt.clim(mind, maxd)
    plt.title(f'Low Res {i}')
    plt.grid(False)
    # sxr
    plt.subplot(133)
    plt.plot(inc_αs[0], vdi[i], 'rs:', label='VDI')
    plt.plot(inc_αs[1], vdc[i], 'gs:', label='VDC')
    plt.plot(inc_αs[2], vde[i], 'bs:', label='VDE')
    plt.plot(inc_αs[3], hor[i], 'ys:', label='HOR')
    plt.xticks([-π/4, -π/8, 0, π/8, π/4], ['-π/4', '-π/8', '0', 'π/8', 'π/4'])
    plt.grid(True)
    plt.legend()
    plt.title(f'SXR {i}')
    plt.show()
    plt.close()

## Test the simulated tomography system on real distributions


In [ ]:
# test of simulation on real distributions
data = np.load(f'data/sxr_real_ds_1000.npz')
print(data.files)
# extract the data
emiss_hr, emiss_lr, vdi, vdc, vde, hor = data['emiss_hr'], data['emiss_lr'], data['vdi'], data['vdc'], data['vde'], data['hor']
print(emiss_hr.shape, emiss_lr.shape, vdi.shape, vdc.shape, vde.shape, hor.shape)
n = emiss_hr.shape[0]

# plot the dataset
N_PLOTS = 20
idxs = np.random.randint(0, n, N_PLOTS)
for i in idxs:
    # now lets calculate the SXR from the high resolution distribution
    sim_vdi = eval_on_fan(emiss_hr[i], fans[0])
    sim_vdc = eval_on_fan(emiss_hr[i], fans[1])
    sim_vde = eval_on_fan(emiss_hr[i], fans[2])
    sim_hor = eval_on_fan(emiss_hr[i], fans[3])
    assert len(sim_vdi) == len(vdi[i]), 'VDI length mismatch'
    assert len(sim_vdc) == len(vdc[i]), 'VDC length mismatch'
    assert len(sim_vde) == len(vde[i]), 'VDE length mismatch'
    assert len(sim_hor) == len(hor[i]), 'HOR length mismatch'

    plt.figure(figsize=(15,6))
    mind, maxd = np.min(emiss_hr[i]), np.max(emiss_hr[i])
    # distribution
    plt.subplot(231)
    plt.scatter(RRH, ZZH, c=emiss_hr[i], s=6)
    plt.axis('equal')
    plt.xlim(R0, R1), plt.ylim(Z0, Z1)
    plt.colorbar()
    plt.clim(mind, maxd)
    plt.title(f'Distribution {i}')
    plt.grid(False)
    # subgrid
    plt.subplot(232)
    plt.scatter(RRL, ZZL, c=emiss_lr[i], s=3)
    plt.axis('equal')
    plt.xlim(R0, R1), plt.ylim(Z0, Z1)
    plt.colorbar()
    plt.clim(mind, maxd)
    plt.title(f'Low Res {i}')
    plt.grid(False)
    # plot the SXR real vs simulated
    plt.subplot(233) # vdi
    plt.plot(inc_αs[0], vdi[i], 'rs:', label='VDI Real')
    plt.plot(inc_αs[0], sim_vdi, 'r-', label='VDI Sim')
    plt.xticks([-π/16, -π/32, 0, π/32, π/16], ['-π/16', '-π/32', '0', 'π/32', 'π/16'])
    plt.grid(True)
    plt.legend()
    plt.title(f'VDI {i}')
    plt.subplot(234) # vdc
    plt.plot(inc_αs[1], vdc[i], 'gs:', label='VDC Real')
    plt.plot(inc_αs[1], sim_vdc, 'g-', label='VDC Sim')
    plt.xticks([-π/8, -π/16, 0, π/16, π/8], ['-π/8', '-π/16', '0', 'π/16', 'π/8'])
    plt.grid(True)
    plt.legend()
    plt.title(f'VDC {i}')
    plt.subplot(235) # vde
    plt.plot(inc_αs[2], vde[i], 'bs:', label='VDE Real')
    plt.plot(inc_αs[2], sim_vde, 'b-', label='VDE Sim')
    plt.xticks([-π/8, -π/16, 0, π/16, π/8], ['-π/8', '-π/16', '0', 'π/16', 'π/8'])
    plt.grid(True)
    plt.legend()
    plt.title(f'VDE  {i}')
    plt.subplot(236) # hor
    plt.plot(inc_αs[3], hor[i], 'ys:', label='HOR Real')
    plt.plot(inc_αs[3], sim_hor, 'y-', label='HOR Sim')
    plt.xticks([-π/4, -π/8, 0, π/8, π/4], ['-π/4', '-π/8', '0', 'π/8', 'π/4'])
    plt.grid(True)
    plt.legend()
    plt.title(f'HOR {i}')

    plt.show()
    plt.close()